In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd 
from skimage.data import imread
import matplotlib.pyplot as plt
import cv2
from rbox.data_utils import *
import os
print(os.listdir("input"))

['train_empty_img_id.npy', 'sample_submission.csv', 'val_empty_img_id.npy', 'train_ship_segmentations.csv', 'train_non_empty_img_id.npy', 'test_ship_segmentations_bbox.csv', 'train', 'train_ship_segmentations_bbox.csv', 'test_ship_segmentations.csv', 'val_non_empty_img_id.npy']


In [ ]:
train = os.listdir('input/train')
print(len(train))

test = os.listdir('input/test')
print(len(test))

In [ ]:
submission = pd.read_csv('input/sample_submission.csv')
submission.head()

In [ ]:
masks = pd.read_csv('input/train_ship_segmentations_bbox.csv')
masks.head(100)

In [ ]:
from collections import defaultdict
imgsz = defaultdict(list)
bad_img = []
for ImageId in masks['ImageId'].tolist():
    try:
        img = imread('input/train/' + ImageId)
    except:
        bad_img.append(ImageId)
        continue
    imgsz[img.shape[0]].append(ImageId)
    

In [ ]:
from collections import defaultdict
import traceback
imgsz_test = defaultdict(list)
bad_img_test = []
for ImageId in masks['ImageId'].tolist():
    try:
        img = imread('input/test/' + ImageId)
    except:
        bad_img_test.append(ImageId)
        print traceback.format_exc()
        continue
    imgsz_test[img.shape[0]].append(ImageId)

In [ ]:
ImageId = '002fdcf51.jpg'

img = imread('input/train/' + ImageId)
img_masks = masks.loc[masks['ImageId'] == ImageId, 'EncodedPixels'].tolist()

# Take the individual ship masks and create a single mask array for all ships
all_masks = full_rle_decode([img_masks[1]], (768, 768))

fig, axarr = plt.subplots(1, 3, figsize=(15, 40))
axarr[0].axis('off')
axarr[1].axis('off')
axarr[2].axis('off')
axarr[0].imshow(img)
axarr[1].imshow(all_masks)
axarr[2].imshow(img)
axarr[2].imshow(all_masks, alpha=0.4)
plt.tight_layout(h_pad=0.1, w_pad=0.1)
plt.show()

In [ ]:
all_masks_img = all_masks
all_masks_img = all_masks_img.astype(np.uint8)

x,y,w,h = cv2.boundingRect(all_masks_img)
rect1 = cv2.rectangle(img.copy(),(x,y),(x+w,y+h),(0,255,0),2)

_,contours,_ = cv2.findContours(all_masks_img, 1, 1)
rect = cv2.minAreaRect(contours[0])
(x,y),(w,h), a = rect
box = cv2.boxPoints(rect)
box = np.int0(box)
rect2 = cv2.drawContours(img.copy(),[box],0,(0,0,255),2)
'''
rows,cols = all_masks_img.shape
M = cv2.getRotationMatrix2D((x,y),(180 + a)%180,1)
dst = cv2.warpAffine(all_masks_img, M, (rows,cols))
#x,y,w,h = cv2.boundingRect(dst)
#rect3 = cv2.rectangle(rect2.copy(),(x,y),(x+w,y+h),(0,255,0),2)

_,contours,_ = cv2.findContours(dst, 1, 1)
rect = cv2.minAreaRect(contours[0])
box = cv2.boxPoints(rect)
box = np.int0(box)
rect3 = cv2.drawContours(rect2.copy(),[box],0,(0,0,255),2)
''' 
#x,y,w,h = np.int0([x,y,w,h])
#rect3 = cv2.rectangle(rect2.copy(),(x-w/2,y-h/2),(x+w/2,y+h/2),(0,255,0),2)

x,y,w,h = cv2.boundingRect(all_masks_img)
rect3 = cv2.rectangle(rect2.copy(),(x,y),(x+w,y+h),(0,255,0),2)


fig, axarr = plt.subplots(1, 3, figsize=(15, 40))
axarr[0].axis('off')
axarr[1].axis('off')
axarr[2].axis('off')
axarr[0].imshow(img)
axarr[1].imshow(rect2)
axarr[2].imshow(rect3)
plt.tight_layout(h_pad=0.1, w_pad=0.1)
plt.show()

In [ ]:
ImageId = 'ccc5df19d.jpg'

img = imread('input/train/' + ImageId)
img_masks = masks.loc[masks['ImageId'] == ImageId, 'EncodedPixels'].tolist()

ImageId_ne = 'fff67c31a.jpg'

img_ne = imread('input/train/' + ImageId_ne)
img_masks_ne = masks.loc[masks['ImageId'] == ImageId_ne, 'EncodedPixels'].tolist()

# Take the individual ship masks and create a single mask array for all ships
all_masks = full_rle_decode(img_masks_ne, (768, 768))

for i in range(768):
    for j in range(768):
        if all_masks[i,j] == 1:
            img[i, j] = img_ne[i, j]

fig, axarr = plt.subplots(1, 3, figsize=(15, 40))
axarr[0].axis('off')
axarr[1].axis('off')
axarr[2].axis('off')
axarr[0].imshow(img)
axarr[1].imshow(all_masks)
axarr[2].imshow(img_ne)
#axarr[2].imshow(img_masks_ne, alpha=0.4)
plt.tight_layout(h_pad=0.1, w_pad=0.1)
plt.show()